In [7]:
import tensorflow as tf
import numpy as np
import cv2 as cv
import pandas as pd
from tqdm import tqdm

from datetime import datetime
import glob
import math
import matplotlib.pyplot as plt


from PIL import Image
import random as python_random
#import seaborn as sns
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import auc, accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.utils import shuffle
import sys
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from keras.applications.resnet import ResNet50, preprocess_input
#from tensorflow.keras.mixed_precision import experimental as mixed_precision
# pip install image-classifiers==1.0.0b1
#from classification_models.tfkeras import Classifiers
# More information about this package can be found at https://github.com/qubvel/classification_models



In [49]:
input_prediction_df = pd.read_csv('saved_models\eval_data\eval_20221117-170236.csv')
input_prediction_df

,0,1,2
0,0.104029,8.283899e-01,0.067581
1,0.141493,7.826040e-01,0.075903
2,0.184859,5.231151e-03,0.809910
3,0.873188,1.472394e-02,0.112088
4,0.180899,1.869610e-03,0.817232
...,...,...,...
11633,0.005997,3.275743e-06,0.994000
11634,0.000180,8.034383e-06,0.999812
11635,0.003633,8.633947e-07,0.996366
11636,0.020479,1.117117e-04,0.979409


In [ ]:
input_df = test_df

true_logits = pd.DataFrame()
loss_log = pd.DataFrame()

In [8]:
df_meta = pd.read_csv('data\mimic_iv_csv\df_race_3class_1117.csv')
train_size, test_size, val_size = df_meta['dataset'].value_counts()
print(df_meta['dataset'].value_counts())
print('Test size: ', test_size)

# Show the model architecture
#print(model.summary())
data_dir = 'data/imgs_race'
test_dir = f'{data_dir}/test'
validate_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_batch_size = 64
HEIGHT = 256
WIDTH = 256
test_batches = validate_gen.flow_from_directory(
                                directory= test_dir,
                                classes = None, # means automatically infer the label from subdir
                                class_mode = 'categorical', # white, black, asian
                                target_size=(HEIGHT, WIDTH),
                                shuffle=False,
                                # seed=seed, 
                                batch_size= test_batch_size
)

train    35635
val      11715
test     11638
Name: dataset, dtype: int64
Test size:  11715
Found 11638 images belonging to 3 classes.


In [40]:
y_batches = []
i = 0
for i in tqdm(range(182)): # 183 is the number of batches
    x_batch, y_batch = test_batches[i]
    y_batches.append(y_batch)

In [41]:
y_batch.shape

(54, 3)

In [45]:
y = np.vstack(y_batches)
y.shape

(11638, 3)

In [46]:
#np.save('data/label_race_test.npy', y)

In [47]:
y = np.load('data/label_race_test.npy')

In [ ]:
def stat_calc(input_prediction_df, ground_truth = y):
    #ground_truth = input_df.race
    pathology_array=[
        'ASIAN',
        'BLACK',
        'WHITE'
        ]
    class_array=[
        'ASIAN',
        'BLACK',
        'WHITE'
        ]
    i=0
    auc_array = []
    for i in range(input_prediction_df.shape[0]):
        truth = 
        pred = input_prediction_df[i]
        AUC = roc_auc_score(truth, pred)
    for pathology in class_array:
    
        #new_truth = (ground_truth.str.contains(pathology)).apply(int)
        new_truth = ground_truth
        input_prediction_val = input_prediction_df[i]
        val = input_prediction_val
        AUC = roc_auc_score(new_truth, val)
        true_logits.insert(i, i, new_truth, True)
        auc_array.append(AUC)
        i += 1
        
    progress_df = pd.DataFrame({'Study':class_array, 'AUC':auc_array})
    print(progress_df)

In [57]:
input_prediction_df.argmax(axis=1).shape

AttributeError: 'DataFrame' object has no attribute 'argmax'

In [60]:
pred = input_prediction_df.to_numpy()
pred

array([[1.0402870e-01, 8.2838994e-01, 6.7581370e-02],
       [1.4149313e-01, 7.8260404e-01, 7.5902930e-02],
       [1.8485852e-01, 5.2311514e-03, 8.0991030e-01],
       ...,
       [3.6328046e-03, 8.6339470e-07, 9.9636626e-01],
       [2.0479380e-02, 1.1171170e-04, 9.7940890e-01],
       [2.3128456e-01, 2.6012356e-03, 7.6611420e-01]])

In [62]:
y_pred = np.argmax(pred, axis = 1)
y_test = np.argmax(y, axis = 1)
y_pred

array([1, 1, 2, ..., 2, 2, 2], dtype=int64)

In [92]:
def evaluate_model(y_test, y_pred, printout = False):
    # test set
    #y_pred = model.predict(X_test) 
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    roc = [] #roc_auc_score(y_test, y_pred)
    stats_test = [acc, precision, recall, f1, roc]
    if printout:
        print(f'Test: \nAccuracy: {acc}\nPrecision: {precision}\nRecall: {recall}\nF1: {f1}\nAUC: {roc}\n')
    
    
    df_metrics = pd.DataFrame(data= {'Test': stats_test},  
                        index=['Accuracy', 'Precision', 'Recall', 'F1-score','AUC score'])
    return df_metrics

In [93]:
evaluate_model(y_test, y_pred)

,Test
Accuracy,0.81294
Precision,0.595789
Recall,0.578234
F1-score,0.577509
AUC score,[]


In [98]:
target_names = ['Asian', 'Black', 'White']
print (classification_report(test_batches.classes, y_pred, target_names=target_names))

              precision    recall  f1-score   support

       Asian       0.13      0.23      0.17       404
       Black       0.78      0.59      0.68      2657
       White       0.88      0.91      0.89      8577

    accuracy                           0.81     11638
   macro avg       0.60      0.58      0.58     11638
weighted avg       0.83      0.81      0.82     11638



In [99]:
print (classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

       Asian       0.13      0.23      0.17       404
       Black       0.78      0.59      0.68      2657
       White       0.88      0.91      0.89      8577

    accuracy                           0.81     11638
   macro avg       0.60      0.58      0.58     11638
weighted avg       0.83      0.81      0.82     11638

